In [1]:
# pip install gensim


Детальное описание

vector_size: Количество измерений векторного представления слова.

window: Максимальное количество слов между текущим и прогнозируемым словом в предложении.

min_count: Минимальное количество раз, которое должно встретиться слово, чтобы оно было учтено.

workers: Количество ядер вашего компьютера, используемых для обучения.

sg: Алгоритм обучения: 1 для Skip-gram; 0 для CBOW.

In [2]:
with open("data.txt", 'r', encoding='utf-8') as file:
    sentences = [line.strip().split() for line in file]


In [3]:
from gensim.models import Word2Vec

# Обучение модели CBOW
model_cbow = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Обучение модели Skip-gram
model_skipgram = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)


ModuleNotFoundError: No module named 'gensim'

In [ ]:
vector_word = model_cbow.wv['king']  # для CBOW
vector_word_skipgram = model_skipgram.wv['king']  # для Skip-gram


In [ ]:
similar_words = model_cbow.wv.most_similar('king', topn=5)
similar_words

[('queen.', 0.14595060050487518),
 ('royal', 0.04157734289765358),
 ('She', 0.03476494550704956),
 ('royal.', 0.01915227621793747),
 ('is', 0.016126055270433426)]

## Word2Vec: Математическое Основание
1. Основные понятия:
Word Embedding — это преобразование слов в векторы чисел. Эти векторы отражают семантические отношения между словами на основе их совместного использования в текстах.

2. Архитектуры:
CBOW (Continuous Bag of Words): Прогнозирует целевое слово на основе его контекста.
Skip-gram: Прогнозирует контекст на основе целевого слова.
3. Функция потерь:
Целью Word2Vec является минимизация функции потерь, которая основана на вероятности слова появиться в определенном контексте.

Для Skip-gram:

На конкретном фрагменте текста приведем модель зависимости между центральным словом и контекстными словами: **картинка**

Для каждой позиции $t=1,\ldots,T$ в заданном поле текста необходимо спрогнозировать контекстные  слова в окне фиксированного размера $m$ по заданному центральному слову $w_{j}$. Нашей задачей является предсказать возможные контекстные слова в поле текста с высокой долей вероятности. Целевая функция $J(\theta)$ называется функцией затрат или потерь и представляет собой среднюю отрицательную логарифмическую вероятность:  
$$J(\theta) = -\frac{1}{T} \sum_{t=1}^{T} \sum_{-m \leq j \leq m, j \neq 0} \log p(w_{t+j} | w_t; \theta)$$

где $\theta$ --- все переменные, подлежащие оптимизации. 

Далее необходимо определить вероятность появления конкретного слова в контексте по заданному центральному слову. Поскольку у нас имеется векторное представление для каждого из слов, то мы можем вычислить вероятность исключительно с точки зрения векторного представления слова. Каждому слову присваивается два вектора слов $v_{w}$, $u_{w}$, где $v_{w}$ --- вектор слов, когда заданное слово используется в качестве центрального слова, $u_{w}$ --- вектор слов, когда заданное слово используется в качестве контекстного слова. Данный прием используется исключительно для упрощения построения векторов слов и оптимизации решения.   то величина вероятности появления контекстного слова с учетом центрального слова  $p(O |C)$ определяется с помощью softmax:

$$p(O | C) = \frac{\exp(u_{O}^T \cdot v_{C})}{\sum_{w=1}^{W} \exp(u_w^T \cdot v_{C})},$$ 
$u_{O}^T \cdot v_{C}$ --- скалярное произведение векторов. 
Отметим, что чем больше значение скалярного произведения векторов 
$$u^T v = u,~ v = \sum_{i=1}^{n} u_{i}v_{i},$$
тем большее значение вероятности мы получаем. Таким образом, скалярное произведение выступает в роли естесственной меры сходства между словами. То есть, если у двух некоторых слов скалярное произведение принимает наибольшее значение, то это означает что данные слова наиболее схожи. 

Приведем пример softmax функции $\mathbb{R}^(n) \rightarrow (0,1)^n: $
$$softmax(x_{i})=\frac{\exp(x_{i})}{\sum_{j=1}^{n}exp(x_{j})}=p_{i}.$$

Функция softmax отображает произвольные значения $x_{i}$ в распределение вероятностей $p_{i}$: 
"max" --- поскольку увеличивает вероятность наибольшего $x_{i}$, "soft" --- поскольку в любом случае присваивает некоторую вероятность меньшему $x_{i}$. 

Нам необходимо максимизировать вероятность контекстных слов, окружающих некоторое центральное слово $w_{j}$.
4. Оптимизация:
Word2Vec использует метод стохастического градиентного спуска для минимизации функции потерь.

5. Negative Sampling:
Чтобы ускорить процесс обучения, вместо того чтобы обновлять все веса в модели, обновляются только небольшое подмножество весов. Это достигается путем выборки "негативных" примеров.

6. Семантические отношения:
Векторные представления слов, полученные с помощью Word2Vec, могут быть использованы для определения отношений между словами. Например, векторное представление слова "король" минус "мужчина" плюс "женщина" приближенно равно векторному представлению слова "королева".

Ассоциативные Модели:
С помощью Word2Vec можно создавать ассоциативные модели для нахождения слов-синонимов и предложения смысловых ассоциаций. Векторное представление позволяет определить, какие слова "ближе" друг к другу в векторном пространстве, что может указывать на их семантическую близость.

Заключение:
Word2Vec — это мощный инструмент для создания векторных представлений слов, которые могут отражать семантические отношения между словами. Эти представления могут быть использованы для многих задач обработки естественного языка, таких как классификация текста, машинный перевод и рекомендательные системы.

## Две основные архитектуры Word2Vec: Skip-gram и CBOW (Continuous Bag of Words).

### 1. Skip-gram

Описание:

- Целью Skip-gram является предсказание контекста (окружающих слов) для данного слова.

- На входе: одно слово.

- На выходе: векторы для каждого из окружающих слов.

Математика:

Для каждого слова $w_t$ в тренировочном корпусе, модель пытается предсказать окружающие слова в окне заданного размера 
n, например, если n = 2, модель предсказывает 2 слова перед и 2 слова после $w_t$.

Преимущества:
- Хорошо работает с небольшим объемом данных.
- Часто лучше выявляет редкие слова.

### 2. CBOW (Continuous Bag of Words)
Описание:
- Целью CBOW является предсказание текущего слова, основываясь на его контексте (окружающих словах).
- На входе: окружающие слова.
- На выходе: вектор для центрального слова.
Математика:
Для каждого слова $w_t$ в тренировочном корпусе, модель использует окружающие слова в окне заданного размера n для предсказания $w_t$ 

Преимущества:
- Обычно требует меньше времени на обучение по сравнению с Skip-gram.
- Хорошо работает на больших корпусах.


Заключение:

Обе модели имеют свои преимущества, и выбор между Skip-gram и CBOW зависит от конкретной задачи и доступных данных. В целом, Skip-gram часто используется для меньших корпусов и когда интерес к редким словам велик, в то время как CBOW подходит для больших корпусов.

## 1. Упрощенный word2vec:
Простая реализация word2vec с использованием библиотеки Torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
import numpy as np

# Токенизация данных
sentences = [["he", "is", "the", "king"],
             ["the", "king", "is", "royal"],
             ["she", "is", "the", "royal", "queen"]]

# Создание словаря
word2idx = defaultdict(lambda: len(word2idx))
for sentence in sentences:
    for word in sentence:
        word2idx[word]

idx2word = {idx: word for word, idx in word2idx.items()}

# Подготовка данных
data = []
for sentence in sentences:
    for i, word in enumerate(sentence):
        for j in range(-2, 3):
            if j != 0 and 0 <= i + j < len(sentence):
                data.append((word2idx[word], word2idx[sentence[i + j]]))

# Параметры
embedding_dim = 5
vocab_size = len(word2idx)

# Модель Skip-Gram
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        out = self.linear(embeds)
        log_probs = torch.log_softmax(out, dim=1)
        return log_probs

# Обучение модели
model = SkipGram(vocab_size, embedding_dim)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):
    total_loss = 0
    for context, target in data:
        model.zero_grad()
        context_var = torch.tensor([context], dtype=torch.long)
        log_probs = model(context_var)
        loss = criterion(log_probs, torch.tensor([target], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

# Получение вектора для слова 'king'
vector = model.embeddings(torch.tensor([word2idx['king']], dtype=torch.long))
print(vector)


Epoch 0, Loss: 70.11177253723145
Epoch 10, Loss: 59.26341676712036
Epoch 20, Loss: 55.53136330842972
Epoch 30, Loss: 53.54115575551987
Epoch 40, Loss: 52.22018963098526
Epoch 50, Loss: 51.236520409584045
Epoch 60, Loss: 50.4513783454895
Epoch 70, Loss: 49.800948560237885
Epoch 80, Loss: 49.25467890501022
Epoch 90, Loss: 48.796350836753845
tensor([[ 0.1622, -0.0047, -0.3847,  1.9700, -0.6172]],
       grad_fn=<EmbeddingBackward0>)


## 2. Word2Vec с использованием gensim:

In [ ]:
from gensim.models import Word2Vec

# Подготовка данных
sentences = [["he", "is", "the", "king"],
             ["the", "king", "is", "royal"],
             ["she", "is", "the", "royal", "queen"]]

# Обучение модели
model = Word2Vec(sentences, vector_size=5, window=5, min_count=1, workers=4)
model.train(sentences, total_examples=len(sentences), epochs=100)

# Получение векторного представления для слова "king"
vector = model.wv['king']
print(vector)


[-0.0363861   0.0575484   0.02003219 -0.16554193 -0.18924105]
